In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!nvcc --version

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b703hni1
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-b703hni1
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=a578709af172fdb514bd01dd0398eb8eeb1d313d6f64438c5cd1c2bab6ba946d
  Stored in directory: /tmp/pip-ephem-wheel-cache-su47zg5f/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


### 1 - Sum of elements in an array

In [ ]:
%%cu
#include <bits/stdc++.h>

#define outc(x) cout<<x<<" "
#define outcn(x) cout<<x<<endl;
#define out(x) cout<<#x " - "<<x<<" "
#define outn(x) cout<<#x " - "<<x<<endl
#define fl(i,n) for(int i=0;i<n;i++)

using namespace std;
using namespace std::chrono;


__global__ void Arr_sum(int *c_a, int *c_n, int *c_sum){
    int tid = threadIdx.x;
    if(tid < (*c_n)){
        atomicAdd(c_sum,c_a[tid]);
    }
}
int main(){
    ios_base::sync_with_stdio(0), cin.tie(0), cout.tie(0);
    srand(chrono::high_resolution_clock::now().time_since_epoch().count());
    int n = 15,sum=0;
    int a[n];
    int *c_a, *c_n, *c_sum;
    fl(i,n){
        a[i] = rand()%10;
        outc(a[i]);
    }
    outcn("");

    cudaMalloc((void**)&c_a,n*sizeof(int));
    cudaMalloc((void**)&c_n,sizeof(int));
    cudaMalloc((void**)&c_sum,sizeof(int));

    cudaMemcpy(c_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(c_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(c_sum,&sum,sizeof(int),cudaMemcpyHostToDevice);

    Arr_sum <<<1,n>>>(c_a,c_n,c_sum);

    cudaMemcpy(&sum,c_sum,sizeof(int),cudaMemcpyDeviceToHost);
    
    outn(sum);
    cudaFree(c_a);
    cudaFree(c_n);
    cudaFree(c_sum);
    return 0;
}

7 2 5 5 8 5 1 6 5 4 4 4 9 7 9 
sum - 81



### 2 - Sum of two arrays

In [ ]:
%%cu

#include <bits/stdc++.h>

#define outc(x) cout<<x<<" "
#define outcn(x) cout<<x<<endl;
#define out(x) cout<<#x " - "<<x<<" "
#define outn(x) cout<<#x " - "<<x<<endl
#define fl(i,n) for(int i=0;i<n;i++)

using namespace std;
using namespace std::chrono;

const short N = 10;

__global__ void Vector_Addition( const int *dev_a , const int *dev_b , int *dev_c)
{
      //Get the id of thread within a block
      unsigned short tid = threadIdx.x ;
     
     // check the boundry condition for the threads
      if ( tid < N ) 
            dev_c [tid] = dev_a[tid] + dev_b[tid] ;
//printf("%p \n", &N);

}


int main ()
{

      int Host_a[N], Host_b[N], Host_c[N];

      int *dev_a , *dev_b, *dev_c ;

      cudaMalloc((void **)&dev_a , N*sizeof(int) ) ;
      cudaMalloc((void **)&dev_b , N*sizeof(int) ) ;
      cudaMalloc((void **)&dev_c , N*sizeof(int) ) ;

      for ( int i = 0; i <N ; i++ )
      {
            Host_a[i] = rand()%10 ;
            Host_b[i] = rand()%10 ; 
      }

      cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice);
      cudaMemcpy (dev_b , Host_b , N*sizeof(int) , cudaMemcpyHostToDevice);

      Vector_Addition <<< 1, N  >>> (dev_a , dev_b , dev_c ) ;

      cudaMemcpy(Host_c , dev_c , N*sizeof(int) , cudaMemcpyDeviceToHost);
      outc("A Vector: ");
      fl(i,N){
          outc(Host_a[i]);
      }
      outcn("");
      outc("B Vector: ");
      fl(i,N){
          outc(Host_b[i]);
      }
      outcn("");
      outc("C Vector / sum : ");
      fl(i,N){
          outc(Host_c[i]);
      }
      outcn("");
      cudaFree (dev_a) ;
      cudaFree (dev_b) ;
      cudaFree (dev_c) ;
      // printf("%p \n", &N);

      return 0 ;

}


A Vector:  3 7 3 6 9 2 0 3 0 2 
B Vector:  6 5 5 2 1 7 9 6 6 6 
C Vector / sum :  9 12 8 8 10 9 9 9 6 8 



### 3 - Matrix multiplication

In [ ]:
%%cu

#include <bits/stdc++.h>

#define outc(x) cout<<x<<" "
#define outcn(x) cout<<x<<endl;
#define out(x) cout<<#x " - "<<x<<" "
#define outn(x) cout<<#x " - "<<x<<endl
#define fl(i,n) for(int i=0;i<n;i++)

using namespace std;
using namespace std::chrono;

__global__ void Matrix_mul(int *a,int *b,int *c,int *m,int *n,int *p)
{
    int col=blockIdx.y*blockDim.y+threadIdx.y;
    int row=blockIdx.x*blockDim.x+threadIdx.x;
    int temp=0,i;

    if(row<*m&&col<*p)
    fl(i,*n){
      temp+=a[row*(*n)+i]*b[i*(*p)+col];
    }
    c[row*(*p)+col]=temp;
}

int main()
{
    int m=rand()%5+1,n=rand()%5+1,p=rand()%5+1,i,j;
    int a[m*n],b[n*p],c[m*n];
    int *cuda_a,*cuda_b,*cuda_c,*cuda_m,*cuda_n,*cuda_p;
    printf("Matrix A:\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<n;j++)
        {
            a[i*n+j]=rand()%10;
            printf("%d ",a[i*n+j]);       
        }
        printf("\n");
    }
    printf("\nMatrix B:\n");
    for(i=0;i<n;i++)
    {
        for(j=0;j<p;j++)
        {
            b[i*p+j]=rand()%10;
            printf("%d ",b[i*p+j]);       
        }
        printf("\n");
    }
    printf("\n");

    cudaMalloc((void**)&cuda_a,m*n*sizeof(int));
    cudaMalloc((void**)&cuda_b,n*p*sizeof(int));
    cudaMalloc((void**)&cuda_c,m*p*sizeof(int));
    cudaMalloc((void**)&cuda_m,sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    cudaMalloc((void**)&cuda_p,sizeof(int));

    cudaMemcpy(cuda_a,a,m*n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b,b,n*p*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_m,&m,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_p,&p,sizeof(int),cudaMemcpyHostToDevice);
 
    dim3 threadsPerBlock(m,p);
    dim3 blocksPerGrid(1,1);
        

    Matrix_mul<<<blocksPerGrid,threadsPerBlock>>> (cuda_a,cuda_b,cuda_c,cuda_m,cuda_n,cuda_p);   

    cudaMemcpy(c,cuda_c,m*p*sizeof(int),cudaMemcpyDeviceToHost);
    printf("Resultant Matrix:\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<p;j++)
        {
            printf("%d ",c[i*p+j]);       
        }
        printf("\n");
    }
    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_c);
    cudaFree(cuda_m);
    cudaFree(cuda_n);
    cudaFree(cuda_p);
    return 0;
}

Matrix A:
5 3 
5 6 
2 9 
1 2 

Matrix B:
7 0 9 
3 6 0 

Resultant Matrix:
44 18 45 
53 36 45 
41 54 18 
13 12 9 



### 5 - BFS

Adjacency matrix/list is replaced with compact adjacency list.

V<sub>a</sub> -  Vertices of the graph

H_Va [2i] will have start index in H_Ea, and H_V[2i+1] will have the length, 
(no.of outgoing edges)


E<sub>a</sub> - Edge vertices for all the vertices in the graph. 

In [ ]:
%%cu

#include <bits/stdc++.h>

using namespace std;
using namespace std::chrono;

#define outc(x) cout<<x<<" "
#define outcn(x) cout<<x<<endl;
#define out(x) cout<<#x " - "<<x<<" "
#define outn(x) cout<<#x " - "<<x<<endl
#define fl(i,n) for(int i=0;i<n;i++)
#define outA(v,n) fl(i,n){outc(v[i]);}outcn("");

__global__ void CUDA_BFS_KERNEL(int *C_Va, int *C_Ea, int *C_Fa, int *C_Xa, 
                                int *C_Ca, int *n, int *n_e){
                                    
    int tid = threadIdx.x;

    if(C_Fa[tid] == 1){
        // outc(tid);
        C_Fa[tid] = 0;
        C_Xa[tid] = 1;
        for(int i=C_Va[2*tid];i<C_Va[2*tid]+C_Va[(2*tid)+1];i++){
            int nid = C_Ea[i];
            if(C_Xa[nid] == 0){
                C_Ca[nid] = C_Ca[tid]+1;
                // # if __CUDA_ARCH__>=200
                    //out(i);
                    // outn(C_Ca[nid]);
                // #endif  
                C_Fa[nid] = 1;
                // *flag = 0;
            }
        }
    }                               
}

int main(){
    ios_base::sync_with_stdio(0), cin.tie(0), cout.tie(0);
    srand(chrono::high_resolution_clock::now().time_since_epoch().count());

    int n = 5 + rand()%4;

    int g[n][n];
    outcn("Adj mat:")
    int n_e = 0;
    fl(i,n){
        fl(j,n){
           if(i==j){
               g[i][j] = 0;
           } 
           else{
               g[i][j] = rand()%2;
           }
           if(g[i][j]!=0){
               n_e++;
           }
           outc(g[i][j]);
        }
        outcn("");
    }

    int H_Va[n*2], H_Ea[n_e*2]; 
    // H_Va [2i] will have start index in H_Ea, and H_V[2i+1] will have the length, 
    //(no.of outgoing edges)
    int length = 0;
    fl(i,n){
        int temp = length;
        H_Va[i*2] = length;
        fl(j,n){
            if(g[i][j]!=0){
                H_Ea[length] = j;
                length++;
            }
        }
        H_Va[(2*i)+1] = length - temp;
    }

    int H_Fa[n] = {0}, H_Xa[n] = {0}, H_Ca[n]; 
    fl(i,n){
      H_Ca[i] = INT_MAX;
      H_Fa[i] = 0;
      H_Xa[i] = 0;
    }
    int s = rand()%n, flag = 0, *C_flag;
    outn(s);
    H_Ca[s] = 0;
    H_Fa[s] = 1;

    int *C_Va, *C_Ea, *C_Fa, *C_Xa, *C_Ca;
    int sz = sizeof(int);
    cudaMalloc((void**)&C_Va,2*n*sz);
    cudaMemcpy(C_Va, H_Va, 2*sz*n, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&C_Ea,2*n_e*sz);
    cudaMemcpy(C_Ea, H_Ea, 2*sz*n_e, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&C_Fa,n*sz);
    cudaMemcpy(C_Fa, H_Fa, sz*n, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&C_Xa,n*sz);
    cudaMemcpy(C_Xa, H_Xa, sz*n, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&C_Ca,n*sz);
    cudaMemcpy(C_Ca, H_Ca, sz*n, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&C_flag,sz);
    int *C_n, *C_ne;
    cudaMalloc((void**)&C_n,sz);
    cudaMemcpy(C_n, &n, sz, cudaMemcpyHostToDevice);

    cudaMalloc((void**)&C_ne,sz);
    cudaMemcpy(C_ne, &n_e, sz, cudaMemcpyHostToDevice);
    outcn("Va:");
    outA(H_Va,2*n);
    outcn("Ea:");
    outA(H_Ea,H_Va[(2*n)-2]+H_Va[(2*n)-1]);
    outcn("Ca:");
    outA(H_Ca,n);

    while(1){
        int flag = 0;
        CUDA_BFS_KERNEL <<<1,n>>>(H_Va,H_Ea,H_Fa,H_Xa,H_Ca,C_n,C_ne);
        cudaDeviceSynchronize();
        cudaMemcpy(H_Fa, C_Fa, sz*n, cudaMemcpyDeviceToHost);

        fl(i,n){
            if(H_Fa[i]==1){
                flag=1;
                break;
            }
        }
        if(flag==0){
            break;
        }
        outcn("Cost:");
        outA(H_Ca,n);
        outn(flag);

    }
    /* do{
        flag = 0;
        // cudaMemcpy(C_flag,&flag,sz,cudaMemcpyHostToDevice);
        CUDA_BFS_KERNEL <<<1,n>>>(H_Va,H_Ea,H_Fa,H_Xa,H_Ca,C_n,C_ne);
        cudaDeviceSynchronize();
        // cudaMemcpy(&flag,C_flag,sz,cudaMemcpyDeviceToHost);
       
        cudaMemcpy(H_Fa, C_Fa, sz*n, cudaMemcpyDeviceToHost);
        outcn("Fa:");
        outA(H_Fa,n);
        fl(i,n){
            if(H_Fa[i]==1){
                flag=1;
                break;
            }
        }
        if(flag==0){
            break;
        }
        outcn("Cost:");
        outA(H_Ca,n);
        outn(flag);
    }while(flag==0);*/

    cudaMemcpy(H_Ca, C_Ca, sz*n, cudaMemcpyDeviceToHost);
    
    outcn("Cost:");
    outA(H_Ca,n);

    cudaFree(C_Ea);
    cudaFree(C_Va);
    cudaFree(C_Ca);
    cudaFree(C_Xa);
    cudaFree(C_Fa);
    cudaFree(C_flag);
    cudaFree(C_n);
    cudaFree(C_ne);

    return 0;
}

In [ ]:
%%cu

#include <bits/stdc++.h>

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/copy.h>
#include <thrust/sort.h>

#define outc(x) cout<<x<<" "
#define outcn(x) cout<<x<<endl;
#define out(x) cout<<#x " - "<<x<<" "
#define outn(x) cout<<#x " - "<<x<<endl
#define fl(i,n) for(int i=0;i<n;i++)
#define outA(v,n) fl(i,n){outc(v[i]);}outcn("");

using namespace std;
using namespace std::chrono;

int main(void)
{
    ios_base::sync_with_stdio(0), cin.tie(0), cout.tie(0);
    srand(chrono::high_resolution_clock::now().time_since_epoch().count());
 
    thrust::host_vector<int> h(10);
    fl(i,10){
        h[i] = rand()%10;
    }
    thrust::device_vector<int> d = h;
    thrust::sort(d.begin(),d.end());
    
    fl(i,10){
        outc(d[i]);
    }
    outcn("");
    return 0;
}

0 2 3 4 5 5 5 6 9 9 

